In [1]:
import pandas as pd #for data reading and manupulation
import numpy as np # for data manupulation
import matplotlib.pyplot as plt # for data Visualization
import seaborn as sns # for data Visualization
%matplotlib inline
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.cross_validation import train_test_split,cross_val_score, cross_val_predict
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import hamming_loss
from sklearn.feature_extraction.text import TfidfVectorizer

C:\Users\Raheem Nasiru\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv('Enriched_training_nas.csv')
test = pd.read_csv('Enriched_mobile_test_nas.csv')

In [3]:
mob = pd.read_csv('mobilemoney_agents_for_upload_win.csv', encoding='latin')

C:\Users\Raheem Nasiru\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (26,32,33,42,74) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
map_train = pd.read_csv('nas_train')
map_tests = pd.read_csv('nas_test')

In [5]:
map_train['region'].unique()

array(['Arusha', 'Manyara', 'Pwani', 'Shinyanga', 'Kagera', 'Mara',
       'Mbeya', 'Dodoma', 'Mwanza', 'Kilimanjaro', 'Tanga', 'Tabora',
       'Dar es Salaam', 'Singida', 'Iringa', 'Kigoma', 'Morogoro', 'Lindi',
       'Mtwara', 'Ruvuma', 'Rukwa', 'Zanzibar'], dtype=object)

In [6]:
train.head()

,OBJECTID,ID,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8_1,...,Population to polygon size rating for the country,Apportionment confidence for the country,HasData,2016 Total Population,2016 Purchasing Power: Total,2016 Pop 25+/Edu: Primary,2016 Pop 25+/Edu: Secondary,2016 Pop 25+/Edu: University/Related,x,y
0,1,5086,98,2,3,1,1,2,2,0,...,2.834,2.614,1,4142.0,3.386526e+09,1229.0,111.0,11.0,29.811396,-4.460442
1,2,1258,40,1,1,3,5,1,1,1,...,2.834,2.614,1,76383.0,1.421899e+11,4664.0,21664.0,1461.0,39.244871,-6.176438
2,3,331,18,2,4,6,3,2,1,0,...,2.834,2.614,1,109482.0,1.167192e+11,37082.0,4785.0,598.0,37.652798,-6.825702
3,4,6729,50,1,1,3,1,1,1,0,...,2.834,2.614,1,2111.0,2.794528e+09,569.0,144.0,27.0,35.808307,-3.372049
4,5,8671,34,1,1,1,1,2,1,0,...,2.834,2.614,0,0.0,0.000000e+00,0.0,0.0,0.0,31.039095,-7.179645


In [7]:
test.head()

,OBJECTID,ID,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8_1,...,bufferRadii,aggregationMethod,Population to polygon size rating for the country,Apportionment confidence for the country,HasData,2016 Total Population,2016 Purchasing Power: Total,2016 Total Households,x,y
0,1,3669,36,2,2,3,1,1,1,0,...,1.0,BlockApportionment:TZ.Regions,2.834,2.614,1,170.0,1.867756e+08,41.0,34.672094,-8.907215
1,2,5223,27,1,1,3,1,2,1,0,...,1.0,BlockApportionment:TZ.Regions,2.834,2.614,1,561.0,6.552594e+08,132.0,35.997953,-8.204943
2,3,7275,25,2,4,6,5,2,1,0,...,1.0,BlockApportionment:TZ.Regions,2.834,2.614,1,10523.0,9.366909e+09,1776.0,32.407155,-4.100044
3,4,7459,32,1,1,1,1,2,1,0,...,1.0,BlockApportionment:TZ.Regions,2.834,2.614,0,0.0,0.000000e+00,0.0,32.242549,-4.054934
4,5,3547,39,1,1,6,1,2,1,0,...,1.0,BlockApportionment:TZ.Regions,2.834,2.614,0,0.0,0.000000e+00,0.0,31.730506,-2.888644


In [8]:
train.columns

Index(['OBJECTID', 'ID', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8_1',
       'Q8_2', 'Q8_3', 'Q8_4', 'Q8_5', 'Q8_6', 'Q8_7', 'Q8_8', 'Q8_9', 'Q8_10',
       'Q8_11', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17',
       'Q18', 'Q19', 'Latitude', 'Longitude', 'mobile_money', 'savings',
       'borrowing', 'insurance', 'mobile_money_classification', 'ID.1',
       'sourceCountry', 'ENRICH_FID', 'areaType', 'bufferUnits',
       'bufferUnitsAlias', 'bufferRadii', 'aggregationMethod',
       'Population to polygon size rating for the country',
       'Apportionment confidence for the country', 'HasData',
       '2016 Total Population', '2016 Purchasing Power: Total',
       '2016 Pop 25+/Edu: Primary', '2016 Pop 25+/Edu: Secondary',
       '2016 Pop 25+/Edu: University/Related', 'x', 'y'],
      dtype='object')

In [9]:
test.columns

Index(['OBJECTID', 'ID', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8_1',
       'Q8_2', 'Q8_3', 'Q8_4', 'Q8_5', 'Q8_6', 'Q8_7', 'Q8_8', 'Q8_9', 'Q8_10',
       'Q8_11', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17',
       'Q18', 'Q19', 'Latitude', 'Longitude', 'ID.1', 'sourceCountry',
       'ENRICH_FID', 'areaType', 'bufferUnits', 'bufferUnitsAlias',
       'bufferRadii', 'aggregationMethod',
       'Population to polygon size rating for the country',
       'Apportionment confidence for the country', 'HasData',
       '2016 Total Population', '2016 Purchasing Power: Total',
       '2016 Total Households', 'x', 'y'],
      dtype='object')

In [10]:
mobile = train['mobile_money_classification']

In [11]:
train = train.drop(['mobile_money', 'savings', 'borrowing', 'insurance', 'mobile_money_classification'], axis=1)

In [12]:
# storing number of rows as an a variable for later use
len_train=train.shape[0]

In [13]:
test = test.drop(['OBJECTID','ID.1', 'sourceCountry',
       'ENRICH_FID', 'areaType', 'bufferUnits', 'bufferUnitsAlias',
       'bufferRadii', 'aggregationMethod',
       'Population to polygon size rating for the country',
       'Apportionment confidence for the country','2016 Total Households', 'x', 'y'], axis=1)

In [14]:
train = train.drop(['OBJECTID', 'ID.1', 'sourceCountry',
       'ENRICH_FID', 'areaType', 'bufferUnits', 'bufferUnitsAlias',
       'bufferRadii', 'aggregationMethod',
       'Population to polygon size rating for the country',
       'Apportionment confidence for the country','2016 Pop 25+/Edu: Primary', '2016 Pop 25+/Edu: Secondary',
       '2016 Pop 25+/Edu: University/Related', 'x', 'y'], axis=1)

In [15]:
train.head()

,ID,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8_1,Q8_2,...,Q15,Q16,Q17,Q18,Q19,Latitude,Longitude,HasData,2016 Total Population,2016 Purchasing Power: Total
0,5086,98,2,3,1,1,2,2,0,0,...,-1,1,-1,4,4,-4.460442,29.811396,1,4142.0,3.386526e+09
1,1258,40,1,1,3,5,1,1,1,0,...,5,4,4,1,4,-6.176438,39.244871,1,76383.0,1.421899e+11
2,331,18,2,4,6,3,2,1,0,0,...,-1,1,-1,1,1,-6.825702,37.652798,1,109482.0,1.167192e+11
3,6729,50,1,1,3,1,1,1,0,0,...,-1,4,-1,1,4,-3.372049,35.808307,1,2111.0,2.794528e+09
4,8671,34,1,1,1,1,2,1,0,1,...,1,1,-1,1,4,-7.179645,31.039095,0,0.0,0.000000e+00


In [16]:
test.head()

,ID,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8_1,Q8_2,...,Q15,Q16,Q17,Q18,Q19,Latitude,Longitude,HasData,2016 Total Population,2016 Purchasing Power: Total
0,3669,36,2,2,3,1,1,1,0,1,...,5,1,-1,1,4,-8.907215,34.672094,1,170.0,1.867756e+08
1,5223,27,1,1,3,1,2,1,0,1,...,6,1,-1,1,1,-8.204943,35.997953,1,561.0,6.552594e+08
2,7275,25,2,4,6,5,2,1,0,1,...,4,5,-1,1,1,-4.100044,32.407155,1,10523.0,9.366909e+09
3,7459,32,1,1,1,1,2,1,0,1,...,-1,1,-1,4,4,-4.054934,32.242549,0,0.0,0.000000e+00
4,3547,39,1,1,6,1,2,1,0,1,...,1,3,3,1,1,-2.888644,31.730506,0,0.0,0.000000e+00


In [17]:
map_train['ID'].nunique()

7094

In [18]:
map_tests['ID'].nunique()

2365

In [19]:
train = train.merge(map_train, on='ID')

In [20]:
test = test.merge(map_tests, on='ID')

In [21]:
#coalescing the two columns for data treatment
data=train.append(test)
data=data.reset_index()
data=data.drop('index',axis=1)

In [22]:
data['latitude2'] = np.round(data.Latitude)
data['longitude2'] = np.round(data.Longitude)

#data['distance'] = distance(**data[['Latitude', 'Longitude', 'latitude2', 'longitude2']])

In [23]:
# Provided function
import numpy as np
def haversine(lon1, lat1, lon2, lat2, earth_radius=6367):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = earth_radius * c
    return km

In [24]:
def haversine_est(lon1, lat1, lon2, lat2):
    R = 6371.0 
    """
    Estimation with equirectangular distance approximation. 
    Since the distance is relatively small, you can use the equirectangular distance approximation. 
    This approximation is faster than using the Haversine formula. 
    So, to get the distance from your reference point (lat1/lon1) to the point your are testing (lat2/lon2),
    use the formula below. 
    Important Note: you need to convert all lat/lon points to radians:
    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    x = (lon2 - lon1) * np.cos( 0.5*(lat2+lat1) )
    y = lat2 - lat1
    km = R * np.sqrt( x*x + y*y )
    return km

In [25]:
data['distance'] = data[['latitude2', 'longitude2', 'Latitude','Longitude']].apply(lambda x: haversine(x[1], x[0], x[3], x[2]), axis=1)

In [26]:
def dummy_manhattan_distance(lon1, lat1, lon2, lat2):
    a = haversine(lon1, lat1, lon2, lat2)
    b = haversine(lon1, lat1, lon2, lat2)
    return a + b

In [27]:
data['distance_duumy'] = data[['latitude2', 'longitude2', 'Latitude','Longitude']].apply(lambda x: dummy_manhattan_distance(x[1], x[0], x[3], x[2]), axis=1)

In [28]:
def bearing_array(lon1, lat1, lon2, lat2):
    AVG_EARTH_RADIUS = 6371  # in km
    lng_delta_rad = np.radians(lon2 - lon1)
    lon1, lat1, lon2, lat2 = map(np.radians, (lon1, lat1, lon2, lat2))
    y = np.sin(lng_delta_rad) * np.cos(lat2)
    x = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(lng_delta_rad)
    return np.degrees(np.arctan2(y, x))

In [29]:
data['distance_bearing'] = data[['latitude2', 'longitude2', 'Latitude','Longitude']].apply(lambda x: bearing_array(x[1], x[0], x[3], x[2]), axis=1)

In [30]:
data['Q8_1_7'] = data['Q8_1'] + data['Q8_7']

In [31]:
data['Q8_9_10'] = data['Q8_10'] + data['Q8_9']

In [32]:
data['qs'] = data['Q8_2'] + data['Q8_3'] + data['Q8_4'] + data['Q8_5'] + data['Q8_6']

In [33]:
#data = data.drop('qs', axis=1)

In [34]:
data['Q3'] = data['Q3'].astype('object')

In [35]:
data['lat_lon'] = data['Latitude'] + data['Longitude']

In [36]:
data = data.drop('ID', axis=1)

In [37]:
#data['Dis_Reg'] = data['region'] + ',' + data['district']

In [38]:
data.head()

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8_1,Q8_2,Q8_3,...,region,latitude2,longitude2,distance,distance_duumy,distance_bearing,Q8_1_7,Q8_9_10,qs,lat_lon
0,98,2,3,1,1,2,2,0,0,0,...,Kigoma,-4.0,30.0,55.271127,110.542253,-157.786948,0,1,0,25.350954
1,40,1,1,3,5,1,1,1,0,0,...,Zanzibar,-6.0,39.0,33.414768,66.829536,125.940889,1,0,0,33.068433
2,18,2,4,6,3,2,1,0,0,0,...,Morogoro,-7.0,38.0,42.921155,85.842310,-63.196030,0,1,0,30.827096
3,50,1,1,3,1,1,1,0,0,0,...,Arusha,-3.0,36.0,46.494014,92.988027,-152.782230,0,0,1,32.436258
4,34,1,1,1,1,2,1,0,1,0,...,Rukwa,-7.0,31.0,20.423306,40.846612,167.815988,0,0,1,23.859450


In [39]:
data = pd.get_dummies(data)

In [40]:
data.head()

,Q1,Q2,Q4,Q5,Q6,Q7,Q8_1,Q8_2,Q8_3,Q8_4,...,region_Mtwara,region_Mwanza,region_Pwani,region_Rukwa,region_Ruvuma,region_Shinyanga,region_Singida,region_Tabora,region_Tanga,region_Zanzibar
0,98,2,1,1,2,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,40,1,3,5,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,18,2,6,3,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,50,1,3,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,34,1,1,1,2,1,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0


In [41]:
# Splitting test and train data
train_df=data[:len_train]
test_df=data[len_train:]

In [42]:
# Removing Price from independen t variable data and keeping only price in the other
X = train_df
y= mobile

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [44]:
from sklearn import metrics

In [45]:
import xgboost as xgb

In [94]:
from catboost import CatBoostClassifier

In [47]:
cat = CatBoostClassifier(loss_function='MultiClass')

In [48]:
#gb =XGBClassifier(n_estimators=200, objective='multi:softprob',gamma=0.5,colsample_bylevel=0.7) CatBoostClassifier(loss_function='MultiClass')

In [49]:
for i in range(5):
    cb_model = CatBoostClassifier(iterations=420,
                              learning_rate=0.1,
                              depth=6,loss_function='MultiClass',#,scale_pos_weight=200,
                             l2_leaf_reg=10,
                             bootstrap_type='Bernoulli',
                              subsample=0.7,
                              #leaf_estimation_iterations=10,
                              metric_period=20,
                              #od_type='Iter',
                              #od_wait=45,
                              random_seed=42,
                              allow_writing_files=False)

In [50]:
cb_model.fit(X,y)

0:	learn: -1.2948722	total: 1.06s	remaining: 7m 25s
20:	learn: -0.8286814	total: 22.1s	remaining: 6m 59s
40:	learn: -0.7745317	total: 40.6s	remaining: 6m 15s
60:	learn: -0.7508234	total: 1m	remaining: 5m 55s
80:	learn: -0.7353918	total: 1m 18s	remaining: 5m 29s
100:	learn: -0.7232368	total: 1m 36s	remaining: 5m 5s
120:	learn: -0.7103068	total: 1m 54s	remaining: 4m 42s
140:	learn: -0.6988756	total: 2m 6s	remaining: 4m 9s
160:	learn: -0.6908904	total: 2m 16s	remaining: 3m 40s
180:	learn: -0.6837903	total: 2m 27s	remaining: 3m 14s
200:	learn: -0.6756696	total: 2m 40s	remaining: 2m 55s
220:	learn: -0.6677922	total: 2m 52s	remaining: 2m 35s
240:	learn: -0.6607858	total: 3m 3s	remaining: 2m 16s
260:	learn: -0.6537183	total: 3m 14s	remaining: 1m 58s
280:	learn: -0.6475890	total: 3m 26s	remaining: 1m 41s
300:	learn: -0.6400614	total: 3m 37s	remaining: 1m 26s
320:	learn: -0.6326724	total: 3m 49s	remaining: 1m 10s
340:	learn: -0.6258011	total: 4m 1s	remaining: 55.9s
360:	learn: -0.6205500	total:

In [51]:
y_xg = cb_model.predict_proba(X_test)

In [52]:
preds = cb_model.predict_proba(X)

In [53]:
from sklearn.metrics import log_loss

In [54]:
log_loss(y, preds)

0.60647009648939421

In [97]:
for i in range(5):
    cb_model1 = CatBoostClassifier(iterations=400,
                              learning_rate=0.1,
                              depth=6,loss_function='MultiClass',#,scale_pos_weight=200,
                             l2_leaf_reg=5,
                             bootstrap_type='Bayesian',
                              #subsample=0.7,
                              leaf_estimation_iterations=1,
                              metric_period=20,
                              #od_type='Iter',
                              #od_wait=45,
                              random_seed=1990,
                              allow_writing_files=False)

In [98]:
cb_model1.fit(X, y)

0:	learn: -1.2863727	total: 1.45s	remaining: 9m 38s
20:	learn: -0.8168542	total: 28s	remaining: 8m 25s
40:	learn: -0.7578999	total: 55.7s	remaining: 8m 7s
60:	learn: -0.7362138	total: 1m 18s	remaining: 7m 17s
80:	learn: -0.7222279	total: 1m 47s	remaining: 7m 2s
100:	learn: -0.7100010	total: 2m 16s	remaining: 6m 43s
120:	learn: -0.6988875	total: 2m 41s	remaining: 6m 13s
140:	learn: -0.6888717	total: 3m 11s	remaining: 5m 51s
160:	learn: -0.6817381	total: 3m 34s	remaining: 5m 18s
180:	learn: -0.6741198	total: 3m 54s	remaining: 4m 43s
200:	learn: -0.6672134	total: 4m 18s	remaining: 4m 15s
220:	learn: -0.6605241	total: 4m 41s	remaining: 3m 48s
240:	learn: -0.6543320	total: 5m 2s	remaining: 3m 19s
260:	learn: -0.6484066	total: 5m 25s	remaining: 2m 53s
280:	learn: -0.6432224	total: 5m 50s	remaining: 2m 28s
300:	learn: -0.6365880	total: 6m 11s	remaining: 2m 2s
320:	learn: -0.6323378	total: 6m 35s	remaining: 1m 37s
340:	learn: -0.6275700	total: 7m 2s	remaining: 1m 13s
360:	learn: -0.6243850	tot

In [167]:
for i in range(5):
    cb_model2 = CatBoostClassifier(iterations=450,
                              learning_rate=0.1,
                              depth=6,loss_function='MultiClass',#,scale_pos_weight=200,
                            l2_leaf_reg=10,
                             bootstrap_type='Bernoulli',
                              subsample=0.7,
                              #leaf_estimation_iterations=,
                              metric_period=20,
                              #od_type='Iter',
                              #od_wait=45,
                              random_seed=24,
                              allow_writing_files=False)

In [168]:
cb_model2.fit(X, y)

0:	learn: -1.2923299	total: 1.59s	remaining: 11m 52s
20:	learn: -0.8292113	total: 25.6s	remaining: 8m 42s
40:	learn: -0.7728118	total: 45.8s	remaining: 7m 36s
60:	learn: -0.7479919	total: 1m 5s	remaining: 6m 55s
80:	learn: -0.7332276	total: 1m 24s	remaining: 6m 24s
100:	learn: -0.7204912	total: 1m 43s	remaining: 5m 57s
120:	learn: -0.7088830	total: 2m 3s	remaining: 5m 34s
140:	learn: -0.6972101	total: 2m 23s	remaining: 5m 13s
160:	learn: -0.6885864	total: 2m 42s	remaining: 4m 50s
180:	learn: -0.6811620	total: 3m	remaining: 4m 28s
200:	learn: -0.6728695	total: 3m 19s	remaining: 4m 7s
220:	learn: -0.6664573	total: 3m 38s	remaining: 3m 46s
240:	learn: -0.6595399	total: 3m 57s	remaining: 3m 26s
260:	learn: -0.6517117	total: 4m 17s	remaining: 3m 6s
280:	learn: -0.6441970	total: 4m 36s	remaining: 2m 46s
300:	learn: -0.6389381	total: 5m	remaining: 2m 28s
320:	learn: -0.6328274	total: 5m 27s	remaining: 2m 11s
340:	learn: -0.6277435	total: 5m 47s	remaining: 1m 51s
360:	learn: -0.6227477	total: 

## for test set

In [169]:
test_set_predictions =  cb_model2.predict_proba(test_df)

In [170]:
d = {"ID": test["ID"], 'no_financial_services':test_set_predictions[:, 0], 'other_only':test_set_predictions[:, 1],
    'mm_only':test_set_predictions[:, 2], 'mm_plus':test_set_predictions[:, 3]}
df_predictions = pd.DataFrame(data=d)
df_predictions = df_predictions[["ID", 'no_financial_services','other_only','mm_only','mm_plus']]

In [148]:
d = {'no_financial_services':test_set_predictions[:, 0], 'other_only':test_set_predictions[:, 1],
    'mm_only':test_set_predictions[:, 2], 'mm_plus':test_set_predictions[:, 3]}
df_predictions = pd.DataFrame(data=d)
df_predictions = df_predictions[['no_financial_services','other_only','mm_only','mm_plus']]

In [116]:
#df_predictions = df_predictions.rank()

In [75]:
df_predictions['ID'] = test['ID']

In [76]:
df_predictions = df_predictions[['ID', 'no_financial_services','other_only','mm_only','mm_plus']]

In [171]:
df_predictions 

,ID,no_financial_services,other_only,mm_only,mm_plus
0,3669,0.007967,0.026987,0.186923,0.778124
1,5223,0.000539,0.007621,0.051026,0.940814
2,7275,0.000286,0.001744,0.123701,0.874269
3,7459,0.137329,0.812466,0.020944,0.029262
4,3547,0.000253,0.000818,0.030565,0.968364
5,888,0.082504,0.069427,0.386349,0.461721
6,207,0.001352,0.001792,0.308786,0.688071
7,5413,0.018314,0.025096,0.172294,0.784296
8,5833,0.334078,0.396437,0.078061,0.191425
9,9221,0.000474,0.002897,0.102317,0.894312


In [58]:
#df_predictions = df_predictions.round(3)

In [81]:
df_predictions.to_csv('rank2.csv',index=False)

In [105]:
df_predictions.to_csv('rank3.csv',index=False)

In [172]:
df_predictions.to_csv('rank4.csv',index=False)

In [173]:
a = pd.read_csv('rank2.csv')

In [174]:
b = pd.read_csv('rank3.csv')
c = pd.read_csv('rank4.csv')

In [175]:
a = a.drop('ID', axis=1)
b = b.drop('ID', axis=1)
c = c.drop('ID', axis=1)

In [176]:
stack = (0.7 * a + 0.1 * b + c * 0.2)

In [177]:
stack = stack.round(3)

In [178]:
stack['ID'] = test['ID']

In [179]:
stack.head()

,no_financial_services,other_only,mm_only,mm_plus,ID
0,0.008,0.029,0.238,0.725,3669
1,0.000,0.006,0.049,0.945,5223
2,0.000,0.001,0.145,0.853,7275
3,0.162,0.791,0.017,0.030,7459
4,0.000,0.001,0.029,0.969,3547


In [180]:
stack = stack[['ID', 'no_financial_services','other_only','mm_only','mm_plus']]

In [181]:
stack.to_csv('stk2.csv',index=False)